# What is Docker?

Lets download and find out ;)

[Download Docker](https://download.docker.com/win/stable/InstallDocker.msi)



In [99]:
(require('https')).get("https://download.docker.com/win/stable/InstallDocker.msi", (r) => r.pipe((require('fs')).createWriteStream("InstallDocker.msi")))
(require('https')).get('https://github.com/jpassing/elevate/releases/download/1.0/elevate.zip', (r) => 
    (require('https')).get(r.headers['location'], (r) => r.pipe((require('fs')).createWriteStream("elevate.zip")))
)
0


Install Docker for Windows quietly

In [106]:
$$async$$ = true;
(require('child_process').exec)('powershell -c "Expand-Archive -Force ' + process.cwd() + '\\elevate.zip"', () => 
(require('child_process').exec)(process.cwd() + '\\elevate\\bin\\x86\\Release\\elevate.exe msiexec /i ' + process.cwd() + '\\InstallDocker.msi /qn /L*V! ' + process.cwd() + '\\log.txt', () => 
$$done$$('done')
).stderr.on('data', (d) => console.log(d))
).stderr.on('data', (d) => console.log(d))
0

Create a Docker container with some dev tools:
* selenium (xvfb, x11vnc, novnc, mocha)
* source code (git, act.ecommerce, angular-cli)

Start from the selenium-chrome-debug image

In [55]:
FROM selenium/standalone-chrome-debug

RUN apt-get -qq update
RUN apt-get install -y --fix-missing git curl wget zip unzip vim dos2unix g++ build-essential python net-tools
RUN wget -O - https://deb.nodesource.com/setup_7.x | bash
RUN apt-get install -y nodejs

RUN npm install -g live-server babel-cli

ADD act.ecommerce /home/seluser/act.ecommerce
ADD mobile /home/seluser/mobile
ADD novnc /home/seluser/novnc

WORKDIR /home/seluser/act.ecommerce
RUN npm install
RUN npm run build

WORKDIR /home/seluser/mobile
RUN npm install

WORKDIR /home/seluser/
ADD new_entry.sh /opt/bin/entry_point.sh
RUN chmod a+x /opt/bin/entry_point.sh
RUN dos2unix /opt/bin/entry_point.sh



Create a Docker file and project directory.

In [23]:
/* Your directory structure should look like this
file -What is Docker.ipynb
dir  -selenium-act
file |--Dockerfile
file |--new_entry.sh
dir  |--act.ecommerce (from git)
dir  |--mobile       (git repo in progress)
dir  |--novnc        (https://github.com/novnc/noVNC/archive/master.zip)
*/



Build the Docker image

In [78]:
$$async$$ = true;
(require('child_process').exec)('docker build -t act-selenium ./selenium-act', () => (require('child_process').exec)('docker images', () => $$.done()).stdout.on('data', (d) => console.log(d))).stdout.on('data', (d) => console.log(d));



Run your Docker container

In [82]:
$$async$$ = true;
(require('child_process').exec)('docker stop act-selenium', () => 
(require('child_process').exec)('docker rm act-selenium', () => 
(require('child_process').exec)('docker run --shm-size=2g --name act-selenium -d -p 8888:8888 -p 6080:6080 -p 5900:5900 -p 4444:4444 -p 4200:4200 act-selenium', () => 
(require('child_process').exec)('docker ps', () => 
$$.done()
).stdout.on('data', (d) => console.log(d))
).stderr.on('data', (d) => console.log(d))
).stdout.on('data', (d) => console.log(d))
).stdout.on('data', (d) => console.log(d));



Connect to VNC so we can see the test play out

[Click here to open](http://localhost:6080/vnc.html?password=secret&host=localhost&port=6080&autoconnect=true&resize=downscale&view_only=true)

In [83]:
$$.html(
    '<div style="display:block; width:100%; padding-bottom:75%;position:relative;">' +
    '<iframe id="vnc"' +
        'style="position:absolute;top:0;right:0;bottom:0;left:0;width:100%;height:100%;border:0;"' +
        'src="http://localhost:6080/vnc.html?password=secret&host=localhost&port=6080&autoconnect=true&resize=downscale&view_only=true"></iframe></div>');

Run our first test by executing through docker

In [84]:
$$.async = true;
(require('child_process').exec)('docker exec -t act-selenium npm --prefix /home/seluser/mobile run test', () => $$.done()).stdout.on('data', (d) => console.log(d));
0